In [3]:
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold,KFold,ShuffleSplit
import cntk as c
import matplotlib.pyplot as plt
from keras import backend as c
from keras import layers
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import LSTM
from keras.optimizers import SGD, Adam
from keras import regularizers

In [4]:
x_train= np.loadtxt('C:/Users/Devanshu/Desktop/faltu/data without misslabel/train_test_without_miss_normalize.csv', delimiter=',', dtype='float32')
x_train_label= np.loadtxt('C:/Users/Devanshu/Desktop/faltu/data without misslabel/label_encoded_train.csv', delimiter=',', dtype='float32')
x_test_label= np.loadtxt('C:/Users/Devanshu/Desktop/faltu/data without misslabel/label_encoded_test.csv', delimiter=',', dtype='float32')
print(x_train.shape)
print(x_train_label.shape)
print(x_test_label.shape)

(144766, 40)
(125973, 5)
(18793, 5)


In [34]:
train= x_train[0:125973,:]
test= x_train[125973:,:]
train_label= x_train_label
test_label= x_test_label
print(train.shape, train_label.shape,test.shape, test_label.shape)
#print(train.shape, train_label.shape,test.shape)

(125973, 40) (125973, 5) (18793, 40) (18793, 5)


In [41]:
print(train[1].shape)
train_1= train.reshape(125973,40,1)
test_1=test.reshape(18793,40,1)

(40,)


In [44]:
model= Sequential()
model.add(Dense(units=128, activation=None, input_shape=(40,1),use_bias=True))
model.add(LSTM(64,activation='tanh', recurrent_activation='hard_sigmoid',return_sequences=True))
model.add(MaxPooling1D(pool_size=5, strides=None, padding='valid'))
model.add(LSTM(32, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None , implementation=1 ))
model.add(Dropout(0.2))
#model.add(Flatten())
model.add(Dense(5, activation='softmax'))

adam=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])
print(model.summary())
model.fit(train_1, train_label, epochs=3, batch_size=128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 40, 128)           256       
_________________________________________________________________
lstm_26 (LSTM)               (None, 40, 64)            49408     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 8, 64)             0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dropout_13 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 5)                 165       
Total params: 62,245
Trainable params: 62,245
Non-trainable params: 0
_________________________________________________________________
None
E

In [45]:
scores= model.evaluate(test_1,test_label)
print(scores[1]*100)

18793/18793 [==============================] - 37s 2ms/step
72.82498802872568


Below code is just for reference. Not for use

In [ ]:
model= Sequential()
model.add(Dense(units=128, activation=None, input_shape=(40,1),use_bias=True))
model.add(LSTM(64,activation='tanh', recurrent_activation='hard_sigmoid',return_sequences=True))
model.add(MaxPooling1D(pool_size=5, strides=None, padding='valid'))
model.add(LSTM(32, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None , implementation=1 ))
model.add(Dropout(0.2))
#model.add(Flatten())
model.add(Dense(5, activation='softmax'))

adam=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])
print(model.summary())
model.fit(train_1, train_label, epochs=2, batch_size=None)

In [ ]:
#enc1= Dense(units=128, activation=None, input_shape=(40,1),use_bias=True)
enc1= LSTM(64,activation='tanh',input_shape=(40,1) ,recurrent_activation='hard_sigmoid',return_sequences=True)
enc2= LSTM(32,activation='tanh',recurrent_activation='hard_sigmoid',return_sequences=True)(enc1)
dec1= LSTM(64,activation='tanh',recurrent_activation='hard_sigmoid',return_sequences=True)(enc2)

model1= Model(input_img, dec1)
layer1= Model(input_img, enc2)

adam=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model1.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])

